In [1]:
// Prequisites:
// sudo apt-get install graphviz
// pip install graphviz

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.2.10:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1604877986701)
SparkSession available as 'spark'


In [2]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification._
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.tuning._
import org.apache.spark.ml.evaluation._
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification._
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.tuning._
import org.apache.spark.ml.evaluation._
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row


In [3]:
val schema = StructType(Array(
    StructField("passenger_id", IntegerType, true),
    StructField("survived", IntegerType, true),
    StructField("ticket_class", IntegerType, true),
    StructField("name", StringType, true),
    StructField("sex", StringType, true),
    StructField("age", DoubleType, true),
    StructField("siblings_spouses", IntegerType, true),
    StructField("parents_children", IntegerType, true),
    StructField("ticket", StringType, true),
    StructField("fare", DoubleType, true),
    StructField("cabin_number", StringType, true),
    StructField("embarked", StringType, true)
))

val raw = spark.read.format("csv")
    .option("header","true")
    .option("inferSchema","true")
    .schema(schema)
    .load("titanic.csv")

raw.show(10, false)

+------------+--------+------------+---------------------------------------------------+------+----+----------------+----------------+----------------+-------+------------+--------+
|passenger_id|survived|ticket_class|name                                               |sex   |age |siblings_spouses|parents_children|ticket          |fare   |cabin_number|embarked|
+------------+--------+------------+---------------------------------------------------+------+----+----------------+----------------+----------------+-------+------------+--------+
|1           |0       |3           |Braund, Mr. Owen Harris                            |male  |22.0|1               |0               |A/5 21171       |7.25   |null        |S       |
|2           |1       |1           |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1               |0               |PC 17599        |71.2833|C85         |C       |
|3           |1       |3           |Heikkinen, Miss. Laina                             |fe

schema: org.apache.spark.sql.types.StructType = StructType(StructField(passenger_id,IntegerType,true), StructField(survived,IntegerType,true), StructField(ticket_class,IntegerType,true), StructField(name,StringType,true), StructField(sex,StringType,true), StructField(age,DoubleType,true), StructField(siblings_spouses,IntegerType,true), StructField(parents_children,IntegerType,true), StructField(ticket,StringType,true), StructField(fare,DoubleType,true), StructField(cabin_number,StringType,true), StructField(embarked,StringType,true))
raw: org.apache.spark.sql.DataFrame = [passenger_id: int, survived: int ... 10 more fields]


In [4]:
val data = raw.select("name","ticket_class","sex","age","siblings_spouses","parents_children","ticket","fare","cabin_number","embarked","survived")
    .withColumn("male",when($"sex" === "male",1).otherwise(0))
    .withColumn("hasFamily", when($"siblings_spouses" > 0 || $"parents_children" > 0,1).otherwise(0))
    .where($"embarked".isNotNull)
    .where($"age".isNotNull)
    .select("name","ticket_class","male","age","hasFamily","fare","embarked","survived")

data.show(false)

+-------------------------------------------------------+------------+----+----+---------+-------+--------+--------+
|name                                                   |ticket_class|male|age |hasFamily|fare   |embarked|survived|
+-------------------------------------------------------+------------+----+----+---------+-------+--------+--------+
|Braund, Mr. Owen Harris                                |3           |1   |22.0|1        |7.25   |S       |0       |
|Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |1           |0   |38.0|1        |71.2833|C       |1       |
|Heikkinen, Miss. Laina                                 |3           |0   |26.0|0        |7.925  |S       |1       |
|Futrelle, Mrs. Jacques Heath (Lily May Peel)           |1           |0   |35.0|1        |53.1   |S       |1       |
|Allen, Mr. William Henry                               |3           |1   |35.0|0        |8.05   |S       |0       |
|McCarthy, Mr. Timothy J                                |1      

data: org.apache.spark.sql.DataFrame = [name: string, ticket_class: int ... 6 more fields]


In [5]:
import org.apache.spark.mllib.tree.DecisionTree

val features = Array("ticket_class", "male","age","hasFamily","fare","embarked_index")
val label = "survived"

val si = new StringIndexer()
    .setInputCol("embarked")
    .setOutputCol("embarked_index")

val ve = new VectorAssembler()
    .setInputCols(features)
    .setOutputCol("features")

val dt = new DecisionTreeClassifier()
    .setFeaturesCol("features")
    .setLabelCol("survived")
    .setMaxDepth(4)


val pl = new Pipeline()
    .setStages(Array(si,ve,dt))
    
val model = pl.fit(data)

import org.apache.spark.mllib.tree.DecisionTree
features: Array[String] = Array(ticket_class, male, age, hasFamily, fare, embarked_index)
label: String = survived
si: org.apache.spark.ml.feature.StringIndexer = strIdx_c3bfa332ccee
ve: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_f843be8ddf05, handleInvalid=error, numInputCols=6
dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_1b01743921d9
pl: org.apache.spark.ml.Pipeline = pipeline_7b683ba90725
model: org.apache.spark.ml.PipelineModel = pipeline_7b683ba90725


In [6]:
val dtcm = model.stages(2).asInstanceOf[DecisionTreeClassificationModel]
println(dtcm.toDebugString)

DecisionTreeClassificationModel: uid=dtc_1b01743921d9, depth=4, numNodes=23, numClasses=2, numFeatures=6
  If (feature 1 <= 0.5)
   If (feature 0 <= 2.5)
    If (feature 2 <= 2.5)
     If (feature 0 <= 1.5)
      Predict: 0.0
     Else (feature 0 > 1.5)
      Predict: 1.0
    Else (feature 2 > 2.5)
     Predict: 1.0
   Else (feature 0 > 2.5)
    If (feature 4 <= 20.7875)
     If (feature 2 <= 17.5)
      Predict: 1.0
     Else (feature 2 > 17.5)
      Predict: 0.0
    Else (feature 4 > 20.7875)
     Predict: 0.0
  Else (feature 1 > 0.5)
   If (feature 2 <= 5.5)
    If (feature 0 <= 2.5)
     Predict: 1.0
    Else (feature 0 > 2.5)
     If (feature 4 <= 20.7875)
      Predict: 1.0
     Else (feature 4 > 20.7875)
      Predict: 0.0
   Else (feature 2 > 5.5)
    If (feature 0 <= 1.5)
     If (feature 2 <= 36.25)
      Predict: 1.0
     Else (feature 2 > 36.25)
      Predict: 0.0
    Else (feature 0 > 1.5)
     Predict: 0.0



dtcm: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel: uid=dtc_1b01743921d9, depth=4, numNodes=23, numClasses=2, numFeatures=6


In [7]:
val tree = dtcm.toDebugString
    .replaceAll("feature 0","ticket_class")
    .replaceAll("feature 1","male")
    .replaceAll("feature 2","age")
    .replaceAll("feature 3","hasFamily")
    .replaceAll("feature 4","fare")
    .replaceAll("feature 5","embarked")

println(tree)

DecisionTreeClassificationModel: uid=dtc_1b01743921d9, depth=4, numNodes=23, numClasses=2, numFeatures=6
  If (male <= 0.5)
   If (ticket_class <= 2.5)
    If (age <= 2.5)
     If (ticket_class <= 1.5)
      Predict: 0.0
     Else (ticket_class > 1.5)
      Predict: 1.0
    Else (age > 2.5)
     Predict: 1.0
   Else (ticket_class > 2.5)
    If (fare <= 20.7875)
     If (age <= 17.5)
      Predict: 1.0
     Else (age > 17.5)
      Predict: 0.0
    Else (fare > 20.7875)
     Predict: 0.0
  Else (male > 0.5)
   If (age <= 5.5)
    If (ticket_class <= 2.5)
     Predict: 1.0
    Else (ticket_class > 2.5)
     If (fare <= 20.7875)
      Predict: 1.0
     Else (fare > 20.7875)
      Predict: 0.0
   Else (age > 5.5)
    If (ticket_class <= 1.5)
     If (age <= 36.25)
      Predict: 1.0
     Else (age > 36.25)
      Predict: 0.0
    Else (ticket_class > 1.5)
     Predict: 0.0



tree: String =
"DecisionTreeClassificationModel: uid=dtc_1b01743921d9, depth=4, numNodes=23, numClasses=2, numFeatures=6
  If (male <= 0.5)
   If (ticket_class <= 2.5)
    If (age <= 2.5)
     If (ticket_class <= 1.5)
      Predict: 0.0
     Else (ticket_class > 1.5)
      Predict: 1.0
    Else (age > 2.5)
     Predict: 1.0
   Else (ticket_class > 2.5)
    If (fare <= 20.7875)
     If (age <= 17.5)
      Predict: 1.0
     Else (age > 17.5)
      Predict: 0.0
    Else (fare > 20.7875)
     Predict: 0.0
  Else (male > 0.5)
   If (age <= 5.5)
    If (ticket_class <= 2.5)
     Predict: 1.0
    Else (ticket_class > 2.5)
     If (fare <= 20.7875)
      Predict: 1.0
     Else (fare > 20.7875)
      Predict: 0.0
   Else (age > 5.5)
    If (ticket_class <= 1.5)
     If (age <= 36.25)
      Predi...


# Display using Graphviz

In [8]:
dtcm.write.overwrite.save("model")
spark.read.load("model/data").createOrReplaceTempView("model")

In [9]:
%%python
model = spark.table("model").toPandas().sort_values("id",ignore_index=True)

In [10]:
%%python
model

,id,prediction,impurity,impurityStats,rawCount,gain,leftChild,rightChild,split
0,0,0.0,0.481757,"[424.0, 288.0]",712,0.138801,1,12,"(1, [0.5], -1)"
1,1,1.0,0.372087,"[64.0, 195.0]",259,0.110873,2,7,"(0, [2.5], -1)"
2,2,1.0,0.108077,"[9.0, 148.0]",157,0.005057,3,6,"(2, [2.5], -1)"
3,3,0.0,0.500000,"[1.0, 1.0]",2,0.500000,4,5,"(0, [1.5], -1)"
4,4,0.0,0.000000,"[1.0, 0.0]",1,-1.000000,-1,-1,"(-1, [], -1)"
5,5,1.0,0.000000,"[0.0, 1.0]",1,-1.000000,-1,-1,"(-1, [], -1)"
6,6,1.0,0.097898,"[8.0, 147.0]",155,-1.000000,-1,-1,"(-1, [], -1)"
7,7,0.0,0.496924,"[55.0, 47.0]",102,0.063545,8,11,"(4, [20.7875], -1)"
8,8,1.0,0.493511,"[35.0, 44.0]",79,0.035937,9,10,"(2, [17.5], -1)"
9,9,1.0,0.351240,"[5.0, 17.0]",22,-1.000000,-1,-1,"(-1, [], -1)"


## Display DesicionTree  

In [11]:
%%python

from graphviz import Graph
from graphviz import Digraph

features = ["ticket_class","male","age","hasFamily","fare","embarked"]

g = Digraph(format='png')

# creating nodes
for index, row in model.iterrows():
    label = ""
    if row["split"]["featureIndex"] >= 0:
        label = features[row["split"]["featureIndex"]] + ">" + str(row["split"]["leftCategoriesOrThreshold"][0] )
    else:
        label = "died"
        if row["prediction"] == 1:
            label = "survived"
        
    g.node(str(row["id"]), label=label)

## creating edges
for index, row in model.iterrows():
    if row['leftChild'] >= 0:
        g.edge(str(row["id"]), str(row['leftChild']),label="no")
    if row['rightChild'] >= 0:
        g.edge(str(row["id"]), str(row['rightChild']),label="yes")

g.render('decisiontree_simple')

![Test Image 1](decisiontree_simple.png)

# Display DesicionTree with Impurities

In [12]:
%%python

from graphviz import Graph
from graphviz import Digraph

features = ["ticket_class","male","age","hasFamily","fare","embarked"]

g = Digraph(format='png')

# creating nodes
for index, row in model.iterrows():
    label = ""
    shape = 'rectangle'
    if row["split"]["featureIndex"] >= 0:
        featureName = str(features[row["split"]["featureIndex"]])
        leftThreshold = str(row["split"]["leftCategoriesOrThreshold"][0]) 
        impurity = str(row["impurity"])
        gain = str(row["gain"])
        label = "%s > %s\n Impurity: %s\n Gain: %s" \
            % (featureName, leftThreshold, impurity, gain)

    else:
        label = "died"
        shape="ellipse"
        if row["prediction"] == 1:
            label = "survived"
        
    g.node(str(row["id"]), label=label, shape=shape)

## creating edges
for index, row in model.iterrows():
    if row['leftChild'] >= 0:
        g.edge(str(row["id"]), str(row['leftChild']),label="no")
    if row['rightChild'] >= 0:
        g.edge(str(row["id"]), str(row['rightChild']),label="yes")

g.render('decisiontree_with_impurities')

![Test Image 1](decisiontree_with_impurities.png)

# Calculating Impurities

In [13]:
// Calculating Impurity for male
val male_feature = data.select("male","survived")
    .groupBy("survived","male").count()

male_feature.show()

+--------+----+-----+
|survived|male|count|
+--------+----+-----+
|       1|   0|  195|
|       1|   1|   93|
|       0|   0|   64|
|       0|   1|  360|
+--------+----+-----+



male_feature: org.apache.spark.sql.DataFrame = [survived: int, male: int ... 1 more field]


In [14]:
val dies_survived = male_feature
    .groupBy("male").pivot("survived").sum()
    .select($"male" ,$"0_sum(count)".as("died"),$"1_sum(count)".as("survived"))
    .withColumn("total",$"died"+$"survived")
dies_survived.show()

+----+----+--------+-----+
|male|died|survived|total|
+----+----+--------+-----+
|   1| 360|      93|  453|
|   0|  64|     195|  259|
+----+----+--------+-----+



dies_survived: org.apache.spark.sql.DataFrame = [male: int, died: bigint ... 2 more fields]


$$G=\sum_{j=1}^{C} p(i)*(1-p(i))$$

In [15]:
// if we use male the gini impurity 
dies_survived
    .agg(sum("died").as("died"),sum("survived").as("survived"),sum("total").as("total"))
    .withColumn("p1",$"died"/$"total")
    .withColumn("p2",$"survived"/$"total")
    .withColumn("gini_impurity",$"p1"*(lit(1)-$"p1") + $"p2"*(lit(1)-$"p2"))
    .show

+----+--------+-----+------------------+------------------+-------------------+
|died|survived|total|                p1|                p2|      gini_impurity|
+----+--------+-----+------------------+------------------+-------------------+
| 424|     288|  712|0.5955056179775281|0.4044943820224719|0.48175735386946095|
+----+--------+-----+------------------+------------------+-------------------+

